In [121]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn import preprocessing

In [81]:
data = [[10,20,1],[11,20,1],[20,10,0],[5,10,1],[51,25,0],[1,2,1],[1,4,1],[3,2,0]]
df = pd.DataFrame(data, columns=["X1","X2","Y"] )
X = df[["X1","X2"]].T
Y = df[["Y"]].T

In [205]:
data = load_breast_cancer()
X = np.array(data["data"]).T
Y = np.array(np.matrix(list(data["target"])))
X.shape

(30, 569)

In [238]:
#Normalize the data
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)


In [190]:
layer_dims = [30,7,4,2,1]

In [191]:
def initialize_parameters(layer_dims):
    parameters = {}
    for i in range(1,len(layer_dims)):
        #using Xavier initialization efficient using tanh activation.
        np.random.seed(10)
        parameters["W"+ str(i)] = np.random.randn(layer_dims[i],layer_dims[i-1])*0.01
        parameters["b"+ str(i)] = np.zeros((layer_dims[i],1))
    return parameters            

In [84]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [85]:
def tanh(x):
    num = np.subtract(np.exp(x), np.exp(-x))
    dem = np.sum([np.exp(x), np.exp(-x)], axis = 0)
    
    return np.divide(num,dem)

In [86]:
def relu(x):
    dG = (x>0).astype(int)
    x = np.array(dG)*np.array(x)
    return x    

In [192]:
def get_layer_values(A_prev,W,b, activation):
    Z = np.dot(W,A_prev)+b
    if activation ==1 or "sigmoid":
        A = sigmoid(Z)
    elif activation == 2 or "tanh":
        A = tanh(Z)
    else:
        A = relu(Z)
    linear_cache = (A_prev,W,b)
    activation_cache = Z
    cache = (linear_cache, activation_cache)
    return A, cache

In [193]:
def get_cost(Y,AL):
    m = Y.shape[1]
    cost = -(np.dot(np.log(AL),Y.T)+np.dot(np.log(1-AL),(1-Y).T))*(1/m)
    return cost

In [230]:
def forward_propagation(X, parameters, activation):
    caches = []
    A_prev = X
    L = int(len(parameters)//2)
    for i in range(1,L+1):
        A,cache_temp = get_layer_values(A_prev= A_prev, W= parameters["W"+ str(i)], b = parameters["b"+ str(i)], activation = activation[i])
        caches.append(cache_temp)
        A_prev = A
    cost = get_cost(Y, AL = A)
    cost = np.squeeze(cost)
    AL = A
    return AL, caches, cost

In [222]:
def get_dZ(dA,cache,activation):
    if activation == 1:
        n = 1/(1+np.exp(cache[1]))
        dG = n*(1-n)
        dZ = dA*dG
    elif activation == 2:
        dG = 1-np.square(cache[1])
        dZ = dA*dG
    else:
        dG = (cache[1]>0).astype(int)
        dZ = dA*dG
    return dZ     

In [223]:
def backward_propagation(AL,Y, caches):
    grads = {}
    dAL = -(np.divide(Y,AL)- np.divide(1-Y, 1-AL))
    L = len(caches)
    m = AL.shape[1]
    #Y = Y.reshape(AL.shape)
    for i in reversed(range(L)):
        grads["dW"+ str(i+1)] = np.dot(get_dZ(dA= dAL,cache = caches[i], activation = activation[i+1]),caches[i][0][0].T)/m
        grads["db"+ str(i+1)] = np.mean(get_dZ(dA = dAL,cache = caches[i], activation = activation[i+1]), axis = 1, keepdims = True)
        grads["dA"+ str(i)] = np.dot(caches[i][0][1].T,get_dZ(dA = dAL,cache = caches[i], activation = activation[i+1]))
        dAL = grads["dA"+ str(i)]
    return grads    

In [210]:
def update_parameters(grads, parameters, learning_rate):
    updated_params = parameters.copy()
    L = len(updated_params)
    for i in range(1,((L//2)+1)):
        updated_params["W"+ str(i)] = parameters["W"+ str(i)] - learning_rate*grads["dW"+ str(i)]
        updated_params["b"+ str(i)] = parameters["b"+ str(i)] - learning_rate*grads["db"+ str(i)]
    return updated_params    

In [226]:
def L_layer_model(X,Y,layers_dims, learning_rate ,num_iter , activation, print_cost = False):
    costs = []
    parameters = initialize_parameters(layers_dims)
    for i in range(num_iter):
        AL,caches, cost =forward_propagation(X,parameters, activation)
        grads = backward_propagation(AL, Y, caches)
        parameters = update_parameters(grads, parameters, learning_rate)
        # Print the cost every 100 iterations
        if print_cost and i % 100 == 0 or i == num_iter - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iter:
            costs.append(cost)
    return parameters, costs        
        


In [247]:
params, costs = L_layer_model(X,Y, layers_dims = [30,7,4,2,1], learning_rate = 0.075, num_iter = 1000, activation = [0,2,2,2,1], print_cost = True)

Cost after iteration 0: 0.6918548944250376
Cost after iteration 100: 0.6604344293184737
Cost after iteration 200: 0.6603167960134015
Cost after iteration 300: 0.6603163458550194
Cost after iteration 400: 0.6603163437729267
Cost after iteration 500: 0.6603163434050547
Cost after iteration 600: 0.6603163430427049
Cost after iteration 700: 0.6603163426792723
Cost after iteration 800: 0.6603163423146685
Cost after iteration 900: 0.6603163419488292
Cost after iteration 999: 0.6603163415853682
